<a href="https://colab.research.google.com/github/davidabelin/doubledigits/blob/main/arithmetic_double_digits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Handwritten Arithmetic

In [ ]:
#@title IMPORTS
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sbn

import zipfile
import math
import random as rd
import numpy as np
import pandas as pd
import os, signal
from IPython.display import clear_output

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
#from tensorflow.keras.preprocessing.image import img_to_array, load_img
#from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, InputLayer, Input
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam

# Construct the Input Data

In [ ]:
#Load
mnist = tf.keras.datasets.mnist
(imgs_train, ans_train), (imgs_test, ans_test) = mnist.load_data()

# Knobs
nrows = 2
ncols = 5

N = 10000 # 10000 num training examples
image_size = (28,56)
epochs = 10
batch_size = 100
learning_rate = 0.005

# Data
imgs_train.shape, imgs_test.shape

In [ ]:
im = np.zeros(image_size)
left_index = rd.randint(0,len(ans_train)-1)   
right_index = rd.randint(0,len(ans_train)-1) 

left_max = np.max(imgs_train[left_index])
left_min = np.min(imgs_train[left_index])
left_scaled = (imgs_train[left_index] - left_min)/(left_max - left_min)

right_max = np.max(imgs_train[right_index])
right_min = np.min(imgs_train[right_index])
right_scaled = (imgs_train[right_index] - right_min)/(right_max - right_min)

im[:,8:32] += left_scaled[:,4:28]
im[:,image_size[0]-4:image_size[1]-8] += right_scaled[:,0:image_size[0]-4]

answer = ans_train[left_index]*10 + ans_train[right_index]
print ("Answer:", ans_train[left_index],"* 10 +",ans_train[right_index],"=",answer)
plt.imshow(im.clip(0,1), cmap='gray')

In [ ]:
def scale_array(arr):
    max = np.max(arr)
    min = np.min(arr)
    scaled = (arr - min)/(max - min)
    return scaled
print("Loaded function scale_array")

def doubleDigits(images=imgs_train, answers=ans_train):
    #returns image array, list of answers
    left_index = rd.randrange(0, len(answers))   
    right_index = rd.randrange(0, len(answers))
    
    answer = answers[left_index]*10 + answers[right_index]
    
    # Seperately scale single digit images to (0:1)
    image = np.zeros(image_size)
    half_width = image_size[1]//2
    width = image_size[1]
    left_scaled = scale_array(images[left_index])
    right_scaled = scale_array(images[right_index])

    # Group digits closer to middle
    image[:,8:half_width+4] += left_scaled[:,4:half_width]
    image[:,half_width-4:width-8] += right_scaled[:,0:half_width-4]
       
    return image.clip(0,1), int(answer)              
    
print("Loaded function doubleDigits")

def getDoubleDigits(images=imgs_train,answers=ans_train,how_many=1):
    yy = np.zeros((how_many,),dtype=int)
    xx = np.zeros((how_many, image_size[0], image_size[1]))
    for i in range(how_many):
        dd, ans = doubleDigits(images, answers)
        yy[i] = ans
        xx[i] = dd
        if i%500 == 0:
            print("Loaded:",i,"of",how_many,"examples...")
            clear_output(wait=True)
    print("Loaded:",how_many,"examples.")
    return xx, yy

print("Loaded function getDoubleDigits")

In [ ]:
x, y = getDoubleDigits(imgs_train,ans_train)
idx = rd.randrange(0,len(y))
print('Answer:',y[idx])
plt.imshow(x[idx], cmap='gray')

In [ ]:
######### GENERATE DD TRAINING DATA FOR FIRST MODEL #################
x_test, y_test = getDoubleDigits(imgs_test,ans_test,N//10)

print("Made",N//20,"new double-digit images to test on.")

x_train, y_train = getDoubleDigits(imgs_train,ans_train,N)

print("Made",N,"new double-digit images to train on.")

######################## Add a channels dimension
x_train = x_train[..., np.newaxis].astype("float32") #was tf.newaxis
x_test = x_test[..., np.newaxis].astype("float32")  #was tf.newaxis

####################### TF Datasets for input
train_ds = tf.data.Dataset.from_tensor_slices( (x_train, y_train) )
test_ds = tf.data.Dataset.from_tensor_slices( (x_test, y_test) )

In [ ]:
x_train = x_train[..., np.newaxis].astype("float32") #was tf.newaxis
x_test = x_test[..., np.newaxis].astype("float32")

### Define Functions

In [ ]:
#@title Display learning curves 
def plotLearningCurves(history):
    acc = history['acc']
    val_acc = history['val_acc']

    loss = history['loss']
    val_loss = history['val_loss']

    epochs = range(len(acc))

    plt.plot(epochs, acc)
    plt.plot(epochs, val_acc)
    plt.title('Training and validation accuracy')

#plotLearningCurves(history)

In [ ]:
#@title Functions GUESSING() and GET_RESULTS()
def guessing(n=1,model=CNN,return_image=False):
    answers, guesses, pA, pG = [],[],[],[]
    for count in range(n):
        take1_ds = test_ds.shuffle(N+1).take(1)
        for img, ans in take1_ds:
            ans = ans.numpy()
            img = img.numpy()                  # eg. 28x56x1
        img = img.reshape((1,) + img.shape)    # eg. 1x50x50x1
        guess_set = model.predict(img).flatten()
        #guess = tf.random.categorical( guess_set, num_samples=1 ).numpy().squeeze()
        guess = np.argmax(guess_set)
        
        answers += [ans]
        guesses += [guess]
        pG += [guess_set[guess]]
        pA += [guess_set[ans]]

        print("Answer",ans,"\tGuess",guess, "\tp(A)",round(pA[count],2),"\tp(G)",round(pG[count],2))
        if count%10 == 0:
            print ('Processing...',count,"...")
            clear_output(wait=True)

    if return_image:
        return answers, guesses, pA, pG, img
    else:
        return answers, guesses, pA, pG

def get_results(n=1,m=CNN):
    results = pd.DataFrame(columns=['Answer','Guess','P(A)','P(G)'])
    results['Answer'], results['Guess'], results['P(A)'], results['P(G)'] = guessing(n, m) 
    return results

In [ ]:
#@title Show layer outputs
def show_layers(model, output_model):
    # Take random image from the training set.
    take1_ds = test_ds.shuffle(10001).take(1)
    for img, ans in take1_ds:
        img = img.numpy()
        ans = ans.numpy()
    img = img.reshape((1,) + img.shape)    # np shape (1, 28, 28, 1)
    gue = model.predict(img)
    print("Answer:",ans, "\tGuess:",np.argmax(gue))
    plt.figure(figsize=(3,3))
    plt.imshow(img[0,:,:,0], cmap="binary_r")                 # np shape (28, 28)

    layer_output_maps = output_model.predict( img )
    layer_names = [layer.name for layer in model.layers[1:]]
    for layer_name, layer_map in zip(layer_names, layer_output_maps):
        if len(layer_map.shape) == 4:# and not 'max_pooling' in layer_name:
            n_maps = layer_map.shape[-1]  # number of maps
            if n_maps > 10:
                n_maps = 10
            # Map has shape (1, rows, columns, n_features)
            rows = layer_map.shape[1]
            cols = layer_map.shape[2]
            image_grid = np.zeros((rows, cols * n_maps))
            
            for i in range(n_maps):
                x = layer_map[0, :, :, i]
                x *= 255.0
                image_grid[:, i * cols : (i + 1) * cols] = x
                image_grid[:,i*cols] = 255.
                image_grid[:,i*cols+1] = 0.
                
            scale = 2.           
            plt.figure(figsize=(scale * n_maps, scale))
            plt.title(layer_name)
            plt.grid(False)
            plt.imshow(image_grid, cmap='gray')
 

In [ ]:
answers, guesses, pA, pG, img = guessing(1, model=minimalNN, return_image=True)
print (answers[0], guesses[0])
print (pA[0], pG[0])
plt.imshow(img[0,:,:,0])

# Build models

In [ ]:
def train(model, epochs=10, batch_size=batch_size):
    history = model.fit(x=x_train,
                        y=y_train,
                        batch_size=batch_size,
                        validation_data=(x_test, y_test),
                        epochs=epochs,  
                        verbose=1)
    return history.history

##Baseline model

In [ ]:
###################### baseline model
input_layer = layers.Input(shape=(image_size[0], image_size[1], 1))
x = layers.Flatten()(input_layer)
output_layer = layers.Dense(100, activation='softmax')(x)#
######################### 

####################### Build
minimalNN = Model(input_layer, output_layer, name="MinimalNN")

####################### Compile
minimalNN.compile(loss="sparse_categorical_crossentropy",
              optimizer=Adam(lr=learning_rate),
              metrics=['acc'])

####################### History containers
answers, guesses = [],[]
minimalNN.summary()

In [ ]:
minimalNN.evaluate(x_test,y_test)

In [ ]:
minimal_stats = train(minimalNN,epochs=epochs,batch_size=batch_size)

##CNN

In [ ]:
###################### Build CNN as first model
input_layer = layers.Input(shape=(image_size[0], image_size[1], 1))
x = layers.Conv2D(20, 2, padding='same', activation='relu')(input_layer)
x = layers.AveragePooling2D(2)(x)
x = layers.Conv2D(30, 3, activation='relu')(x) 
x = layers.MaxPooling2D(2)(x)
x = layers.Flatten()(x)
output_layer = layers.Dense(100, activation='softmax')(x)

####################### Build
CNN = Model(input_layer, output_layer, name="CNN")

####################### Compile
CNN.compile(loss="sparse_categorical_crossentropy",
            optimizer=Adam(lr=learning_rate),
            metrics=['acc'])

####################### Layer-outputs model
layer_outputs = [layer.output for layer in CNN.layers[1:]]
CNN_outputs = Model(input_layer, layer_outputs)

####################### History containers
answers, guesses = [],[]
CNN.summary()

In [ ]:
for layer in CNN.layers:
    #layer.trainable = False
    print(layer_name,":",layer.trainable)
CNN.summary()

In [ ]:
CNN.evaluate(x_test,y_test)

In [ ]:
show_layers(CNN, CNN_outputs)

In [ ]:
CNN_stats = train(CNN,epochs=1,batch_size=batch_size)

In [ ]:
show_layers(CNN, CNN_outputs)

In [ ]:
CNN.evaluate(x_test,y_test)

In [ ]:
predicts = [np.argmax(p) for p in CNN.predict(x_test)]
rid = rd.randrange(len(y_test))
print ("Guess:",predicts[rid], "Ans:",y_test[rid])
plt.imshow(x_test[rid].reshape((28,56)), cmap="gray")

## Mixed model

In [ ]:
##################### mixedNN
for layer in CNN.layers:
    layer.trainable = False
    if "flatten" in layer.name: out_layer = layer.name

x = CNN.get_layer(out_layer).output
x = layers.Dense(200, activation='relu')(x)
x = layers.Dropout(0.2)(x)
new_output_layer = layers.Dense(100, activation='softmax')(x)
mixedNN = Model(input_layer, new_output_layer)
mixedNN.compile(loss="sparse_categorical_crossentropy",
                 optimizer=Adam(lr=learning_rate),
                 metrics=['acc'])

####################### Layer-outputs for new model
new_layer_outputs = [layer.output for layer in mixedNN.layers[1:]]
mixed_outputs = Model(input_layer, new_layer_outputs)

mixedNN.summary()

In [ ]:
#for layer in CNN.layers:
#    layer.trainable = True
for layer in mixedNN.layers:
    print (layer.name,":",layer.trainable)

In [ ]:
mixedNN.evaluate(x_test,y_test)

In [ ]:
############################# Train new_model
mixed_stats = train(mixedNN)

In [ ]:
plotLearningCurves(minimal_stats)

In [ ]:
show_layers(mixedNN, mixed_outputs)

In [ ]:
results = get_results(100,mixedNN)
results

In [ ]:
tv = results['Guess'] == results['Answer']
wrongs = results.loc[~tv]
wrongs.sort_values('Answer')

##Build modelX

In [ ]:
plt.imshow(rd.choice(x_train)[:,:,0])

In [ ]:
###### BEST MODEL: NN MODEL ########### 
input_layer = layers.Input(shape=(image_size[0], image_size[1], 1))

########################### This works ###########################
x = layers.Flatten()(input_layer)
x = layers.Dense(300, activation='relu')(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(200, activation='relu')(x) 
x = layers.Dropout(0.2)(x)
output_layer = layers.Dense(100, activation='softmax')(x) 
modelX = Model(input_layer, output_layer)
########################### ^^^^^^^^^^ ###########################

####################### Compile
modelX.compile(loss="sparse_categorical_crossentropy",
              optimizer=Adam(lr=learning_rate),
              metrics=['acc'])
    
####################### Layer-outputs model
#layer_outputsX = [layer.output for layer in modelX.layers[1:]]
#output_modelX = Model(input_layer, layer_outputsX)

####################### History containers
answersX, guessesX = [],[]

####################### Summary
modelX.summary()
#tf.keras.utils.plot_model(modelX,show_shapes=True)

In [ ]:
############################# Train modelX
historyX = train(modelX,epochs=1,batch_size=batch_size)

In [ ]:
acc = historyX['acc']
val_acc = historyX['val_acc']
epochs = len(acc)

plt.plot(range(epochs), acc)
plt.plot(range(epochs), val_acc)
plt.title('Training and validation accuracy')

In [ ]:
#@title Show layer output from MODELX ..?

# Take random image from the training set.
take1_ds = test_ds.shuffle(10001).take(1)
for img, ans in take1_ds:
    img = img.numpy()
    ans = ans.numpy()
img = img.reshape((1,) + img.shape)    # np shape (1, 28, 28, 1)
gue = modelX.predict(img)
print("Answer:",ans, "\tGuess:",np.argmax(gue))
plt.figure(figsize=(3,3))
plt.imshow(img[0,:,:,0])                 # np shape (28, 28)

layer_output_maps = output_modelX.predict( img )
layer_names = [layer.name for layer in modelX.layers[1:]]
for layer_name, layer_map in zip(layer_names, layer_output_maps):
    if "flatten" in layer_name: 
        layer_map = layer_map.reshape(1,image_size[0],image_size[1])
        n_maps = layer_map.shape[0]  # 1 = number of maps

        # Map has shape (1, rows, columns)
        #rows = layer_map.shape[1]
        #cols = layer_map.shape[2]
        #image_grid = np.zeros((rows, cols))
        
        #for i in range(n_maps):
        x = layer_map[0, :, :]
        #x *= 255.0
        #image_grid[:, cols : 2 * cols] = x
            
        scale = 2.           
        plt.figure(figsize=(scale * n_maps, scale))
        plt.title(layer_name)
        plt.grid(False)
        plt.imshow(x)  #(image_grid, cmap='viridis')

In [ ]:
layer_output_maps = output_modelX.predict( img )
layer_names = [layer.name for layer in modelX.layers[1:]]
for layer_name, layer_map in zip(layer_names, layer_output_maps):
    if "flatten" in layer_name:#len(layer_map.shape) == 4:# and not 'max_pooling' in layer_name:
         layer_map = layer_map.reshape(1,28,56)
    print (layer_name, layer_map.shape)

In [ ]:
####################### Layer-outputs model
#layer_outputsX = [layer.output for layer in modelX.layers[1:]]
#output_modelX = Model(input_layer, layer_outputsX)
modelX_neurons = output_modelX.variables[0].numpy().T
modelX_neurons_df = pd.DataFrame(modelX_neurons)
neuronX_df = pd.DataFrame(modelX_neurons[2].reshape((28,56)))
#neuronX_df              

In [ ]:
plt.imshow(neuronX_df)

In [ ]:
nxmin = min(neuronX_df.min())
nxmax = max(neuronX_df.max())
nxmean = np.mean(neuronX_df.mean())
nxstd = np.mean(neuronX_df.std())
print (nxmin,nxmax,nxmean,nxstd)
print ("nxmin*255, nxmax*255", nxmin*255, nxmax*255)

#neuronX_df *= 255.
neuronX_df += nxstd
neuronX_df.clip(0,nxmax)
#neuronX_df *= 255

plt.imshow(neuronX_df)

In [ ]:
modelX.variables[0].shape

In [ ]:
variablesX = modelX.variables[0].numpy()
(vmin,vmax,vmean,vstd) = (variablesX.min(), variablesX.max(), variablesX.mean(), variablesX.std())
(vmin,vmax,vmean,vstd)

In [ ]:
variablesX = np.clip((variablesX + abs(vstd))*255,0,255)
variablesX

In [ ]:
variables = modelX.variables[0].numpy()
num_nodes = variables.shape[1]
num_rows = int(math.ceil(num_nodes / 10.0))
fig, axes = plt.subplots(num_rows, 10, figsize=(40, 2 * num_rows))
for coef, ax in zip(variables.T, axes.ravel()):
    # Weights in coef is reshaped from 1x784 to 28x28.
    coef = coef
    ax.matshow(coef.reshape(image_size), cmap=plt.cm.viridis)
    ax.set_xticks(())
    ax.set_yticks(())
plt.show()

In [ ]:
minimal_results = get_results(100, mixedNN)
resultsCNN = get_results(100, CNN)
resultsX = get_results(100, modelX)
#resultsX

In [ ]:
plt.scatter(minimal_results['Guess'],minimal_results['Answer'])
plt.scatter(resultsCNN['Guess'],resultsCNN['Answer'])
plt.scatter(resultsX['Guess'],resultsX['Answer'])

In [ ]:
tvX = resultsX['Guess'] == resultsX['Answer']
wrongsX = resultsX.loc[~tvX]
wrongsX.sort_values('Answer')

In [ ]:
# "one shot iterator"
answers, guesses, pA, pG, img = guessing(1, model=modelX, return_image=True)
print ("Label:", answers[0], "\tPredict:", guesses[0])
print ("pLabel =",pA[0], "\tpPredict = ", pG[0])
plt.imshow(img[0,:,:,0],cmap="gray")

In [ ]:
def showWeights():
    weights0 = modelX.weights[0].numpy()
    num_nodes = weights0.shape[1]
    num_rows = int(math.ceil(num_nodes / 10.0))
    fig, axes = plt.subplots(num_rows, 10, figsize=(40, 2 * num_rows))
    for coef, ax in zip(weights0.T, axes.ravel()):
        # Weights in coef is reshaped from 1x784 to 28x28.
        ax.matshow(coef.reshape(image_size), cmap=plt.cm.bone)
        ax.set_xticks(())
        ax.set_yticks(())
    plt.show()

In [ ]:
showWeights()